In [1]:
#hide
#default_exp tools.print_table
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Print table

> Prints results obtained in the experiment indicated as argument, across all the runs 

In [2]:
#export
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from IPython.display import display
import argparse

import sys
sys.path.append('.')
sys.path.append('src')

from hpsearch.config.hpconfig import get_experiment_manager, get_path_experiments
import hpsearch.utils.experiment_utils as ut
from hpsearch.tools.metric_visualization import include_best_and_last_experiment
import hpsearch.config.hp_defaults as dflt

In [3]:
#for tests
import pytest
from hpsearch.examples.dummy_experiment_manager import generate_data

## print_table

In [4]:
#export
def print_table (experiments=[-1, -2], path_experiments=None, folder=None, display_all=False, 
                 include_best=False, op=None, metric=None, round_digits=2, compare=True, 
                 compact=0, run_number=0, manager_path=dflt.manager_path):
        
    em = get_experiment_manager (manager_path=manager_path)
    if path_experiments is not None or folder is not None:
        em.set_path_experiments (path_experiments=path_experiments, folder=folder)
    if metric is not None:
        em.key_score = metric
    if op is not None:
        em.op = op
        
    path_experiments = em.path_experiments
        
    df = pd.read_pickle(path_experiments/'experiments_data.pk')

    if display_all:
        display (df[em.key_score])

    metric_column = f'{run_number}_{em.key_score}'
        
    experiments = include_best_and_last_experiment ([metric], experiments=experiments, 
                                                    run_number=run_number, op=em.op)


    df_scores = None
    print ('\n*****************************')
    for e in experiments:
        parameters = ut.get_parameters_columns(df.loc[e:e+1], True)
        print ('\nparameters for %d:' %e)
        display (df.loc[e,parameters])
        print ('scores for all experiments:')
        df_scores = ut.get_experiment_scores(df.loc[[e]], suffix_results='_%s' %em.key_score, remove_suffix=True)
        display(df_scores.round(round_digits))
        print ('score:')
        display (df.loc[e, metric_column])

    df2 = None
    if len(experiments) > 0 and compare:
        parameters = ut.get_parameters_columns(df.loc[experiments], True)
        print ('\ncomparison')
        display (df.loc[experiments, parameters])
        print ('score')
        display (df.loc[experiments, metric_column])

        print ('with unique parameters:')
        _, df2 = ut.get_parameters_unique(df.loc[experiments])
        if compact > 0:
            prev_cols = df2.columns.copy()
            df2, dict_rename = ut.compact_parameters (df2, compact)
            for k, kor in zip(df2.columns, prev_cols):
                print ('{} => {}'.format(k, kor))
        display(df2)

    return df, df2, df_scores

### usage

In [5]:
#export tests.tools.test_print_table
def test_print_table ():
    em = generate_data ('print_table')
    
    print_table (manager_path=em.manager_path)
    
    print_table (compare=True, manager_path=em.manager_path)
    
    print_table (experiments=[2, 6, -2], compare=True, compact=3, manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [6]:
tst.run (test_print_table, tag='dummy')

running test_print_table

*****************************

parameters for 8:


epochs      30
noise      0.1
offset     0.6
rate      0.03
Name: 8, dtype: object

scores for all experiments:


,0,1,2,3,4
8,0.69,0.97,0.95,0.84,0.77


score:


...

,epochs,noise,offset,rate
2,5.0,0.1,0.6,0.03
6,30.0,0.1,0.1,0.03
5,15.0,0.1,0.6,0.03


score


2    0.554331
6    0.393850
5    1.000000
Name: 0_validation_accuracy, dtype: float64

with unique parameters:
Epo => epochs
Off => offset
0ValAcc => 0_validation_accuracy
0TesAcc => 0_test_accuracy
Tim0 => time_0
Dat => date
1ValAcc => 1_validation_accuracy
1TesAcc => 1_test_accuracy
Tim1 => time_1
2ValAcc => 2_validation_accuracy
2TesAcc => 2_test_accuracy
Tim2 => time_2
3ValAcc => 3_validation_accuracy
3TesAcc => 3_test_accuracy
Tim3 => time_3
4ValAcc => 4_validation_accuracy
4TesAcc => 4_test_accuracy
Tim4 => time_4


,Epo,Off,0ValAcc,0TesAcc,Tim0,Dat,1ValAcc,1TesAcc,Tim1,2ValAcc,2TesAcc,Tim2,3ValAcc,3TesAcc,Tim3,4ValAcc,4TesAcc,Tim4
2,5.0,0.6,0.554331,0.827450,0.000233,21:13:19.669841,0.588019,0.866663,0.000222,0.603512,0.734215,0.000219,0.766158,0.773693,0.000230,0.835562,0.921418,0.000244
6,30.0,0.1,0.393850,0.202604,0.000307,21:13:20.477696,0.392414,0.292257,0.000329,0.362720,0.400189,0.000331,0.549044,0.358083,0.000329,0.287793,0.340739,0.000326
5,15.0,0.6,1.000000,0.910991,0.000259,21:13:20.266897,0.858103,0.882976,0.000279,1.000000,0.809882,0.000254,1.000000,0.842953,0.000283,1.000000,1.000000,0.000284


## parse_arguments_and_run

In [7]:
#export
def parse_args(args):
    parser = argparse.ArgumentParser(description='print table') 
    # Datasets
    parser.add_argument('--folder', type=str, default=None, help='name of experiments folder')
    parser.add_argument('--path_experiments', type=str, default=None, help='full path')
    parser.add_argument('-m', '--metric', type=str, default=None, help='metric score')
    parser.add_argument('-e', type=int, nargs='+', default=[-1, -2], help='experiment numbers')
    parser.add_argument('-a', type=bool, default=False)
    parser.add_argument ('-b', '--best', action= "store_true", help='include experiment with best performance (on given run id!!)')
    parser.add_argument ('-r', '--run', type=int, default=0, help='run id') 
    parser.add_argument('-n', '--no_comp', action= "store_true", help='do not perform comparison')
    parser.add_argument('--compact', type=int, default=0, help='compact parameters to this number of characters') 
    parser.add_argument('--op', default=None, type=str)
    parser.add_argument('--round', default=2, type=int, help='round scores to this number of digits')
    parser.add_argument('-p', '--path', default=dflt.manager_path, type=str)
    pars = parser.parse_args(args)
    
    return pars

def parse_arguments_and_run (args):
    
    pars = parse_args(args)

    df, df2, df_scores=print_table (experiments=pars.e, path_experiments = pars.path_experiments, 
                                    folder=pars.folder, 
                                    display_all=pars.a, include_best=pars.best, op=pars.op, 
                                    metric=pars.metric, round_digits=pars.round, 
                                    compare=not pars.no_comp, compact=pars.compact, 
                                    run_number=pars.run, manager_path=pars.path)

def main():
    parse_arguments_and_run (sys.argv[1:])

### usage

In [8]:
#export tests.tools.test_print_table
def test_parse_arguments_and_run ():
    em = generate_data ('parse_arguments_and_run')
    
    args = ['-e', '4', '3',
           '--compact', '3',
           '--round', '3',
           '-p', em.manager_path]
    parse_arguments_and_run (args)
    
    em.remove_previous_experiments (parent=True)

In [9]:
tst.run (test_parse_arguments_and_run, tag='dummy')

running test_parse_arguments_and_run

*****************************

parameters for 4:


epochs      15
noise      0.1
offset     0.3
rate      0.03
Name: 4, dtype: object

scores for all experiments:


,0,1,2,3,4
4,0.732,0.628,0.776,0.821,0.879


score:


...

,epochs,noise,offset,rate
4,15.0,0.1,0.3,0.03
3,15.0,0.1,0.1,0.03


score


4    0.732285
3    0.707854
Name: 0_validation_accuracy, dtype: float64

with unique parameters:
Off => offset
0ValAcc => 0_validation_accuracy
0TesAcc => 0_test_accuracy
Tim0 => time_0
Dat => date
1ValAcc => 1_validation_accuracy
1TesAcc => 1_test_accuracy
Tim1 => time_1
2ValAcc => 2_validation_accuracy
2TesAcc => 2_test_accuracy
Tim2 => time_2
3ValAcc => 3_validation_accuracy
3TesAcc => 3_test_accuracy
Tim3 => time_3
4ValAcc => 4_validation_accuracy
4TesAcc => 4_test_accuracy
Tim4 => time_4


,Off,0ValAcc,0TesAcc,Tim0,Dat,1ValAcc,1TesAcc,Tim1,2ValAcc,2TesAcc,Tim2,3ValAcc,3TesAcc,Tim3,4ValAcc,4TesAcc,Tim4
4,0.3,0.732285,0.924655,0.000281,21:13:22.207040,0.627768,0.481516,0.000272,0.776318,0.722544,0.000261,0.821438,0.683518,0.000251,0.879119,0.679725,0.000279
3,0.1,0.707854,0.346810,0.000257,21:13:21.997027,0.508697,0.419747,0.000275,0.545406,0.439323,0.000264,0.663109,0.428636,0.000269,0.577253,0.319690,0.000268
